In [271]:
cd/Users/choejuhui/Desktop/jupyter/Rhythm-of-Chika-data/Datasets

/Users/choejuhui/Desktop/jupyter/Rhythm-of-Chika-data/Datasets


In [272]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [303]:
position = np.load('Left_DMF_p_3.npy')
position_2 = np.load('Left_UMF_p_3.npy')

In [233]:
position.shape #300프레임, 21개 랜드마크, xyz값

(300, 21, 3)

In [304]:
velocity = np.zeros((1,21,3))
acceleration = np.zeros((1,21,3))

for i in range(len(position)-1):
    current_velocity = position[i+1] - position[i]
    velocity = np.vstack((velocity, current_velocity.reshape(1,21,3)))

velocity = velocity[1:]

for i in range(len(velocity)-1):
    current_acceleration = velocity[i+1] - velocity[i]
    acceleration = np.vstack((acceleration, current_acceleration.reshape(1,21,3)))

acceleration = acceleration[1:]

In [305]:
velocity = np.zeros((1,21,3))
acceleration_2 = np.zeros((1,21,3))

for i in range(len(position_2)-1):
    current_velocity = position_2[i+1] - position_2[i]
    velocity = np.vstack((velocity, current_velocity.reshape(1,21,3)))

velocity = velocity[1:]

for i in range(len(velocity)-1):
    current_acceleration = velocity[i+1] - velocity[i]
    acceleration_2 = np.vstack((acceleration_2, current_acceleration.reshape(1,21,3)))

acceleration_2 = acceleration_2[1:]

## 이상치 제거 1안

In [306]:
#dataframe
d1 = pd.DataFrame(acceleration[:,4,:], columns=['x','y','z']) #4번 랜드마크 DMF
d2 = pd.DataFrame(acceleration[:,8,:], columns=['x','y','z']) #8번 랜드마크
d3 = pd.DataFrame(acceleration[:,13,:], columns=['x','y','z']) #13번 랜드마크
d1_2 = pd.DataFrame(acceleration_2[:,4,:], columns=['x','y','z']) #4번 랜드마크 UMF

d1.plot()
d2.plot()
d3.plot()


In [309]:
d1.describe() #아래 가운데 앞면

,x,y,z
count,298.000000,298.000000,298.000000
mean,0.000004,-0.000007,-0.000014
std,0.004475,0.004343,0.003590
min,-0.014186,-0.013934,-0.013586
25%,-0.002557,-0.002580,-0.002037
50%,0.000067,0.000385,0.000221
75%,0.002748,0.003000,0.002426
max,0.017376,0.010031,0.008361


In [310]:
d1_2.describe() #위 가운데 앞면

,x,y,z
count,298.000000,298.000000,298.000000
mean,-0.000012,-0.000012,-0.000008
std,0.003824,0.004183,0.003031
min,-0.010234,-0.014206,-0.007032
25%,-0.002250,-0.002279,-0.002014
50%,-0.000062,-0.000457,-0.000118
75%,0.002460,0.001833,0.002071
max,0.011971,0.018790,0.008236


In [291]:
#이상치 제거
idx_list=[]
axis = ['x','y','z']
for i in axis:
    q3=d1['%s' % (i)].quantile(0.75)
    q1=d1['%s' % (i)].quantile(0.25)
    iqr=q3-q1
    outlier_top=q3+1.5*iqr
    outlier_bottom=q1-1.5*iqr

    idx=d1[(d1['%s' % (i)]<outlier_bottom) | (d1['%s' % (i)]>outlier_top)].index
    for i in idx:
        idx_list.append(str(i))
    idx_list.append('end')
    #d1=d1.drop(idx)

In [292]:
idx_string = ' '.join(idx_list)
idx_string_split = idx_string.split('end')
x_outlier_str = idx_string_split[0].strip()
y_outlier_str = idx_string_split[1].strip()
z_outlier_str = idx_string_split[2].strip()


z_outlier_str.split(' ')[:-1]

#np.where 안에 넣을 인덱스(int)
x_outlier = list(map(int, x_outlier_str.split(' ')[:-1]))
y_outlier = list(map(int, y_outlier_str.split(' ')[:-1]))
z_outlier = list(map(int, z_outlier_str.split(' ')[:-1]))

In [293]:
x_outlier

[28, 36, 68, 69, 188, 224, 244, 251, 252, 258, 260]

## 이상치 제거 2안

In [294]:
#numpy
d4 = acceleration[:,4,:]
d5 = acceleration[:,8,:]
d6 = acceleration[:,13,:]

In [295]:
d4[39][0]

0.006337940692901611

In [296]:
#이상치 보정
for i in range(len(d4)-1):
    if i in x_outlier:
        d4[i][0] = (d4[i-1][0] + d4[i+1][0])/2 
    if i in y_outlier:
        d4[i][1] = (d4[i-1][1] + d4[i+1][1])/2 
    if i in z_outlier:
        d4[i][2] = (d4[i-1][2] + d4[i+1][2])/2 
        
for i in range(len(d5)-1):
    if i in x_outlier:
        d5[i][0] = (d5[i-1][0] + d5[i+1][0])/2 
    if i in y_outlier:
        d5[i][1] = (d5[i-1][1] + d5[i+1][1])/2 
    if i in z_outlier:
        d5[i][2] = (d5[i-1][2] + d5[i+1][2])/2 
        
for i in range(len(d6)-1):
    if i in x_outlier:
        d6[i][0] = (d6[i-1][0] + d6[i+1][0])/2 
    if i in y_outlier:
        d6[i][1] = (d6[i-1][1] + d6[i+1][1])/2 
    if i in z_outlier:
        d6[i][2] = (d6[i-1][2] + d6[i+1][2])/2 

In [283]:
d4[38][0], d4[39][0], d4[40][0]

(0.0014780163764953613, 0.0032573938369750977, 0.005036771297454834)

In [246]:
d4[38][0], d4[39][0], d4[40][0]

(0.0014780163764953613, -0.02270185947418213, 0.005036771297454834)